In [1]:
!rm -rf ./experiments
!rm -rf ./some/other/path
%config InlineBackend.figure_format = 'retina'

# Basic Usage


`digital-experiments` works straight out of the box:

In [1]:
from digital_experiments import experiment

@experiment
def square(x):
    return x * x

[square(i) for i in range(5)]

[0, 1, 4, 9, 16]

Get all the [Observations](core-api.rst#digital_experiments.core.Observation) from the experiment (these are persisted over multiple python sessions):

In [2]:
square.observations()

[Observation(2024-02-04_08-20-11_161787, {'x': 0} → 0),
 Observation(2024-02-04_08-20-11_528886, {'x': 1} → 1),
 Observation(2024-02-04_08-20-11_539600, {'x': 2} → 4),
 Observation(2024-02-04_08-20-11_549778, {'x': 3} → 9),
 Observation(2024-02-04_08-20-11_559902, {'x': 4} → 16)]

If you have `pandas` installed, you can also use the [to_dataframe](core-api.rst#digital_experiments.core.Experiment.to_dataframe) method:

In [3]:
square.to_dataframe()

,id,result,config.x
0,2024-02-04_08-20-11_161787,0,0
1,2024-02-04_08-20-11_528886,1,1
2,2024-02-04_08-20-11_539600,4,2
3,2024-02-04_08-20-11_549778,9,3
4,2024-02-04_08-20-11_559902,16,4


## Observations

Each [Observation](core-api.rst#digital_experiments.core.Observation) object is a light-weight wrapper around:

- a unique identifier (implemented as a timestamped string)
- the exact configuration (args, kwargs and defaults) used to run the experiment
- the result of the experiment (the return value of the function)
- a dictionary of metadata that internal and user-defined [callback hooks](callbacks-api.rst) can use to store other relevant information

In [4]:
import json

_dict = square.observations()[0]._asdict()
print(json.dumps(_dict, indent=4))

{
    "id": "2024-02-04_08-20-11_161787",
    "config": {
        "x": 0
    },
    "result": 0,
    "metadata": {
        "system": {
            "platform": "macOS-14.2.1-arm64-arm-64bit",
            "machine": "arm64",
            "processor": "arm",
            "system": "Darwin",
            "python_version": "3.8.18",
            "pwd": "/Users/john/projects/digital_experiments/docs/source"
        },
        "pip_freeze": "alabaster==0.7.13\nanyio==4.2.0\nappnope==0.1.3\nargon2-cffi==23.1.0\nargon2-cffi-bindings==21.2.0\narrow==1.3.0\nasttokens==2.4.1\nasync-lru==2.0.4\nattrs==23.2.0\nBabel==2.14.0\nbackcall==0.2.0\nbeautifulsoup4==4.12.3\nbleach==6.1.0\nbumpver==2023.1129\ncertifi==2023.11.17\ncffi==1.16.0\ncharset-normalizer==3.3.2\nclick==8.1.7\ncolorama==0.4.6\ncomm==0.2.1\ncoverage==7.4.0\ndebugpy==1.8.0\ndecorator==5.1.1\ndefusedxml==0.7.1\n-e git+https://github.com/jla-gardner/digital-experiments.git@d15279995c45c78c703e08223580fc75995e2ea1#egg=digital_experiments\ndocut

By default, `digital-experiments` times how long the experiment took, and the exact code that was run. The latter is particularly useful when we're rapdily iterating on an experiment's code, and want to be able to reproduce the results of a previous run. Other useful information is also stored, such as the current `git commit`, details of the python environment and information about the machine the experiment was run on. This ensures a high level of reproducibility and traceability.

Extra timing information can be added to this metadata by using the [time_block](core-api.rst#digital_experiments.time_block) context.

## Backends

By default, `digital-experiments` stores each observation in its own `.pkl` file located at `./experiments/<experiment_name>/<id>.pkl`:

In [5]:
!ls ./experiments/square

2024-02-04_08-20-11_161787.pkl 2024-02-04_08-20-11_549778.pkl
2024-02-04_08-20-11_528886.pkl 2024-02-04_08-20-11_559902.pkl
2024-02-04_08-20-11_539600.pkl


Other backends are available (see the [complete list here](backends-api.rst)), or you can [implement your own](backends-api.rst).

You can also specify the root directory for a given experiment by passing the `root` argument to the [@experiment](core-api.rst#digital_experiments.experiment) decorator, or by setting the `DE_ROOT` environment variable:

In [6]:
from pathlib import Path

@experiment(root=Path("some/other/path"))
def cube(x):
    return x ** 3

cube(4)
!ls ./some/other/path

2024-02-04_08-20-13_600755.pkl


## Artefacts

`digital-experiments` assigns and provides a unique directory on disk per run of an experiment. This can be accessed within an experiment using the [current_dir](core-api.rst#digital_experiments.current_dir) function. Any files saved to this directory during the experiment are available _post hoc_ via the [artefacts](core-api.rst#digital_experiments.core.Experiment.artefacts) function.

In [7]:
from digital_experiments import current_dir

@experiment
def saving_experiment():
    (current_dir() / 'test.txt').write_text('hello world')

saving_experiment()
id = saving_experiment.observations()[0].id
saving_experiment.artefacts(id)

[PosixPath('experiments/saving_experiment/storage/2024-02-04_08-20-13_746886/test.txt')]